In [490]:
from transformers import AutoTokenizer,AutoModelForCausalLM,AutoModelForSequenceClassification, GPT2Tokenizer, GPT2ForSequenceClassification, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd

Initialize the GPT-2 tokenizer and model

In [555]:
tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")
model = AutoModelForCausalLM.from_pretrained("gpt2-medium")

In [517]:
df = pd.read_csv('../data/corpus_turims.csv', encoding='utf8')

In [518]:
df['words'] =  [item.split() for item in df['Respuesta']]

In [519]:
df.head(10)

,Pregunta,Respuesta,words
0,¿Cuál es el lugar turístico más visitado en Ec...,Las Islas Galápagos son el destino más popular...,"[Las, Islas, Galápagos, son, el, destino, más,..."
1,¿Qué se puede hacer en las Islas Galápagos?,"En las Islas Galápagos, puedes hacer buceo, sn...","[En, las, Islas, Galápagos,, puedes, hacer, bu..."
2,¿Cuál es el volcán más alto de Ecuador?,El volcán Chimborazo es el pico más alto de Ec...,"[El, volcán, Chimborazo, es, el, pico, más, al..."
3,¿Qué ciudades coloniales son famosas en Ecuador?,Quito y Cuenca son conocidas por sus hermosos ...,"[Quito, y, Cuenca, son, conocidas, por, sus, h..."
4,¿Cuál es el monumento más famoso en Quito?,La Basílica del Voto Nacional es uno de los mo...,"[La, Basílica, del, Voto, Nacional, es, uno, d..."
5,¿Dónde se encuentra la mitad del mundo en Ecua...,"La Mitad del Mundo está cerca de Quito, en la ...","[La, Mitad, del, Mundo, está, cerca, de, Quito..."
6,¿Cuáles son las playas más hermosas de la cost...,"Las playas de Salinas, Montañita y Ayangue son...","[Las, playas, de, Salinas,, Montañita, y, Ayan..."
7,¿Qué parque nacional es conocido por su selva ...,El Parque Nacional Yasuní es famoso por su bio...,"[El, Parque, Nacional, Yasuní, es, famoso, por..."
8,¿Qué animales se pueden ver en el Parque Nacio...,"En Yasuní, puedes ver monos aulladores, jaguar...","[En, Yasuní,, puedes, ver, monos, aulladores,,..."
9,¿Qué actividades se pueden hacer en el Parque ...,"En el Parque Nacional Cajas, puedes hacer send...","[En, el, Parque, Nacional, Cajas,, puedes, hac..."


In [520]:
dataset = (load_dataset('csv', data_files='../data/corpus_turims.csv', encoding='utf8', split='train').train_test_split(train_size=.8, test_size=.2))

In [521]:
dataset = (Dataset.from_pandas(df, split='train').train_test_split(train_size=.8, test_size=.2))

In [522]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Pregunta', 'Respuesta', 'words'],
        num_rows: 47
    })
    test: Dataset({
        features: ['Pregunta', 'Respuesta', 'words'],
        num_rows: 12
    })
})

In [506]:
def subfinder(words_list, answer_list):
    matches = []
    start_indices = []
    end_indices = []
    for idx, i in enumerate(range(len(words_list))):
        if words_list[i] == answer_list[0] and words_list[i : i + len(answer_list)] == answer_list:
            matches.append(answer_list)
            start_indices.append(idx)
            end_indices.append(idx + len(answer_list) - 1)
    if matches:
        return matches[0], start_indices[0], end_indices[0]
    else:
        return None, 0, 0

In [507]:
def encode_dataset(examples, max_length=512):
    questions = examples["Pregunta"]
    words = examples["words"]
    answers = examples["Respuesta"]

    # encode the batch of examples and initialize the start_positions and end_positions
    encoding = tokenizer(questions, words, max_length=max_length, padding="max_length", truncation=True)
    start_positions = []
    end_positions = []

    # loop through the examples in the batch
    for i in range(len(questions)):
        cls_index = encoding["input_ids"][i].index(tokenizer.cls_token_id)

        # find the position of the answer in example's words
        words_example = [word.lower() for word in words[i]]
        answer = answers[i]
        match, word_idx_start, word_idx_end = subfinder(words_example, answer.lower().split())

        if match:
            # if match is found, use `token_type_ids` to find where words start in the encoding
            token_type_ids = encoding["token_type_ids"][i]
            token_start_index = 0
            while token_type_ids[token_start_index] != 1:
                token_start_index += 1

            token_end_index = len(encoding["input_ids"][i]) - 1
            while token_type_ids[token_end_index] != 1:
                token_end_index -= 1

            word_ids = encoding.word_ids(i)[token_start_index : token_end_index + 1]
            start_position = cls_index
            end_position = cls_index

            # loop over word_ids and increase `token_start_index` until it matches the answer position in words
            # once it matches, save the `token_start_index` as the `start_position` of the answer in the encoding
            for id in word_ids:
                if id == word_idx_start:
                    start_position = token_start_index
                else:
                    token_start_index += 1

            # similarly loop over `word_ids` starting from the end to find the `end_position` of the answer
            for id in word_ids[::-1]:
                if id == word_idx_end:
                    end_position = token_end_index
                else:
                    token_end_index -= 1

            start_positions.append(start_position)
            end_positions.append(end_position)

        else:
            start_positions.append(cls_index)
            end_positions.append(cls_index)

    encoding["start_positions"] = start_positions
    encoding["end_positions"] = end_positions

    return encoding

In [523]:
def tokenize_function(examples):
    output_dict = tokenizer([" ".join(x) for x in examples["Pregunta"]],[" ".join(x) for x in examples["words"]],  max_length=512, truncation=True)
    return output_dict 

In [524]:
block_size = 1024


def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [469]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["Pregunta"], truncation=True)

    labels = []
    for i, label in enumerate(examples[f"Respuesta"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [445]:
tokenizer.pad_token = tokenizer.eos_token

In [444]:
tokenizer.add_special_tokens({'eos_token': '[PAD]'})

0

In [556]:
tokenized_datasets = dataset.map(
    tokenize_function, 
    batched=True,
    remove_columns=dataset["train"].column_names,
    )

Map:   0%|          | 0/47 [00:00<?, ? examples/s]

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

In [526]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 47
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 12
    })
})

In [557]:
lm_dataset = tokenized_datasets.map(
    group_texts, 
    batched=True,
    remove_columns=tokenized_datasets["train"].column_names,
    )

Map:   0%|          | 0/47 [00:00<?, ? examples/s]

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

In [534]:
lm_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1
    })
})

In [558]:
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [548]:
training_args = TrainingArguments(
    output_dir="./my_awesome_model",
    learning_rate=2e-5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [549]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
)

In [559]:
trainer.train()

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 2.3149704933166504, 'eval_runtime': 2.922, 'eval_samples_per_second': 4.107, 'eval_steps_per_second': 0.684, 'epoch': 1.0}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 2.237075090408325, 'eval_runtime': 2.937, 'eval_samples_per_second': 4.086, 'eval_steps_per_second': 0.681, 'epoch': 2.0}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 2.2130930423736572, 'eval_runtime': 2.915, 'eval_samples_per_second': 4.117, 'eval_steps_per_second': 0.686, 'epoch': 3.0}
{'train_runtime': 119.031, 'train_samples_per_second': 1.185, 'train_steps_per_second': 0.151, 'train_loss': 2.398239347669813, 'epoch': 3.0}


TrainOutput(global_step=18, training_loss=2.398239347669813, metrics={'train_runtime': 119.031, 'train_samples_per_second': 1.185, 'train_steps_per_second': 0.151, 'train_loss': 2.398239347669813, 'epoch': 3.0})

In [560]:
trainer.save_model()

In [138]:
prompt = "Cuál es el lugar turístico más visitado en Ecuador"

In [561]:
model = AutoModelForCausalLM.from_pretrained("./my_awesome_model")

In [574]:
input_question = "¿Que puedo hacer en quito?"
input_tokens = tokenizer(input_question, padding=True, truncation=True, return_tensors="pt")
output = model.generate(input_tokens.input_ids,max_new_tokens=50)

# Decode and print the generated answer
generated_answer = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Answer:", generated_answer)

AttributeError: 'GPT2LMHeadModel' object has no attribute 'predict'

In [271]:
import evaluate

accuracy = evaluate.load("accuracy")

In [274]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [571]:
def get_response_from_corpus(user_input, corpus_df, tokenizer):
    """Buscar la respuesta en el corpus basada en la pregunta del usuario."""
    user_tokens = tokenizer.encode(user_input.lower(), add_special_tokens=False)

    for index, row in corpus_df.iterrows():
        pregunta = row['Pregunta'].lower()
        respuesta = row['Respuesta']
        for token in user_tokens:
            if token in tokenizer.encode(pregunta, add_special_tokens=False):
                return respuesta

    return None

In [568]:

corpus_df = pd.read_csv('../data/corpus_turims.csv', encoding='utf-8')

In [570]:
for index, row in corpus_df.iterrows():
    print(row)

Pregunta     ¿Cuál es el lugar turístico más visitado en Ec...
Respuesta    Las Islas Galápagos son el destino más popular...
Name: 0, dtype: object
Pregunta           ¿Qué se puede hacer en las Islas Galápagos?
Respuesta    En las Islas Galápagos, puedes hacer buceo, sn...
Name: 1, dtype: object
Pregunta               ¿Cuál es el volcán más alto de Ecuador?
Respuesta    El volcán Chimborazo es el pico más alto de Ec...
Name: 2, dtype: object
Pregunta      ¿Qué ciudades coloniales son famosas en Ecuador?
Respuesta    Quito y Cuenca son conocidas por sus hermosos ...
Name: 3, dtype: object
Pregunta            ¿Cuál es el monumento más famoso en Quito?
Respuesta    La Basílica del Voto Nacional es uno de los mo...
Name: 4, dtype: object
Pregunta     ¿Dónde se encuentra la mitad del mundo en Ecua...
Respuesta    La Mitad del Mundo está cerca de Quito, en la ...
Name: 5, dtype: object
Pregunta     ¿Cuáles son las playas más hermosas de la cost...
Respuesta    Las playas de Salinas, Montañi

In [572]:
def setup_chatbot():
   # Load the pre-trained GPT-2 model and tokenizer

   # Load your corpus from the CSV file
   corpus_df = pd.read_csv('../data/corpus_turims.csv', encoding='utf-8')

   print("Chatbot: ¡Hola! Soy un chatbot. ¿En qué puedo ayudarte hoy?")

   while True:
      user_input = input("Tú: tienes alguna consulta el día de hoy?")
      if user_input.lower() == 'exit':
         print("Chatbot: Hasta luego. ¡Que tengas un buen día!")
         break

      # Buscar la respuesta en el corpus basada en la pregunta del usuario
      response = get_response_from_corpus(user_input, corpus_df, tokenizer)

      if response:
         print("Chatbot:", response)
      else:
         print("Chatbot: Lo siento, no tengo información sobre eso en mi corpus.")

In [573]:
setup_chatbot()

Chatbot: ¡Hola! Soy un chatbot. ¿En qué puedo ayudarte hoy?
Chatbot: Las Islas Galápagos son el destino más popular en Ecuador.
Chatbot: Hasta luego. ¡Que tengas un buen día!
